In [2]:
Float = Sys.WORD_SIZE == 64 ? Float64 : Float32;

In [9]:
mutable struct BloomFilter
    setSize::Int             # size of the original set
    falsePositiveProb::Float # probability of false positive
    hashFuncsNumber::Int     # number of hash functions
    hashFunc::Function       # hash function to encode the probabilistic "belongs to" relation
    bitArray::Vector{Bool}   # "the" bloom filter
    BloomFilter(n, p, hF) = optimalBloomFilter(n, p, hF)
end

function optimalBloomFilter(setSize::Int, falsePositiveProb::Float, hashFunc::Function)
    n = setSize
    p = falsePositiveProb
    
    hF = hashFunc
    
    m = -n*log(p)/log(2)^2
    k = -log(2,p)
    
    bitArray = falses(trunc(Int, m) + 1)
    
    return BloomFilter(n, p, trunc(Int, k), hF, bitArray)
end

function add!(bf::BloomFilter, str::String)
    for i in bf.hashFunc(bf, str)
        bf.bitArray[i] = true
    end
end

# this result is conclusive only if it is false, 
# when true the element may or may not be in the set
function belongsTo(bf::BloomFilter, str::String)::Bool
    isElement = true
    
    for i in bf.hashFunc(bf, str)
        isElement &= bf.bitArray[i]
    end
    
    return isElement
end
;

In [10]:
function simplestHash(bf::BloomFilter, str::String)::Vector{Int}
    m, = size(bf.bitArray)
    k = bf.hashFuncsNumber
    ret = zeros(k)
    
    for i=1:k
        ret[i] = (hash(str, hash(i)) % m) + 1
    end
    
    return ret
end

function kmHash(bf::BloomFilter, str::String)::Vector{Int}
    h1 = hash("1"*str)
    h2 = hash("2"*str)
    
    k = bf.hashFuncsNumber
    ret = zeros(k)
    
    for i=1:k
        ret[i] = (h1 + i*h2 % m) + 1
    end
    
    return ret
end
;

In [11]:
# theoretical upper bound for false positive probability 
# of a Bloom filter
function worstTheoreticalFP(bf::BloomFilter)::Float
    m,= size(bf.bitArray)
    n = bf.setSize
    k = bf.hashFuncsNumber
    
    return (1 - exp(-k*(n + 0.5)/(m-1)))^k
end
;

In [6]:
using CSV

In [18]:
csv_filename = "Shakespeare_data.csv"
csv_file = CSV.File(csv_filename)

n = length(csv_file)
p = 0.01

simpleHashFunc = simplestHash
kmHashFunc = kmHash

simpleBF = BloomFilter(n, p, simpleHashFunc)
kmBF = BloomFilter(n, p, kmHashFunc)
;

MethodError: MethodError: no method matching BloomFilter(::Int64, ::Float64, ::Int64, ::typeof(simplestHash), ::BitArray{1})
Closest candidates are:
  BloomFilter(::Any, ::Any, ::Any) at In[9]:7

In [8]:
typeof(simpleBF)

UndefVarError: UndefVarError: simpleBF not defined

Useful Links
---

* https://sagi.io/2017/07/bloom-filters-for-the-perplexed/
* https://github.com/johnmyleswhite/BloomFilters.jl/blob/master/src/bloom-filter.jl
* https://en.wikipedia.org/wiki/Bloom_filter#Optimal_number_of_hash_functions
* https://www.eecs.harvard.edu/~michaelm/postscripts/rsa2008.pdf
* https://github.com/JuliaLang/IJulia.jl/issues/636
* http://juliadata.github.io/CSV.jl/stable/

* http://www.jesse-anderson.com/2011/10/a-few-million-monkeys-randomly-recreate-every-work-of-shakespeare/